### Properties

We saw that we can define "bare" attributes in classes and instances.

```python
class MyClass:
    def __init__(self, language):
        self.language = language
obj = MyClass('Python')

print(obj.language)  # direct access to 'language' attribute
obj.language = 'Java'  # direct access to 'language' attribute
```

in many languages direct access to attributes is highly discouraged <br>
Instead the convention is to make the attribute private, and create public getter and setter methods <br>
Although we don't have private attributes in Python, we could write it this way: <br>

```python
class MyClass:
    def __init__(self, language):
        self._language = language

    def get_language(self):
        return self._language

    def set_language(self, value):
        self._language = value 
```

What code would you rather to write?
```python
print(obj.language)
obj.language = 'Java'
```

or 

```python
print(obj.get_language())
obj.set_language('Java')
```

There are some good reasons why we might want to approach attributes using this programming style <br>
> provides __control__ on how an attribute's value is set and returned

#### issue
If you start with a class that provides direct access to the language attribute, and later need to 
change it to use accessor methods, you will change the interface of the class <br>
any code that uses the attribute directly: 
```
obj.language = 'Java'
```
will need to be refactored to use the accessor methods instead:
```
obj.set_language('Java')
```

#### Python has a Solution
We can use the __property__ class to define properties in a class:

```python
class MyClass:
    def __init__(self, language):
        self.language = language

class MyClass:
    def __init__(self, language):
        self._language = language
    
    def get_language(self):
        return self._language

    def set_language(self, value):
        self._language = value

    language = property(fget=get_language, fset=set_language)
    
m = MyClass('Python')
m.language = 'Java'
print(m.language)
```

> We changed an attribute to a property without changing the class interface! 

### The property Class
property is a class (type)
    constructor has a few parameters:
   + fget: specifies the function to use to get instance property value
   + fset: specifies the function to use to set the instance property value
   + fdel: specifies the function to call when deleting the instance property
   + doc:  a string representing the docstring for the property

> In general we start with plain attributes, and if later we need to change to a 
property we can easily do so using the property class without changing the 
interface

Let's start with a simple example and a bare attribute:

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name

So this class has a single instance **attribute**, `name`.

In [ ]:
p = Person('Alex')

And we can access and modify that attribute using either dotted notation or the `getattr` and `setattr` methods:

In [ ]:
p.name

In [ ]:
getattr(p, 'name')

In [ ]:
p.name = 'John'

In [ ]:
p.name

In [ ]:
setattr(p, 'name', 'Eric')

In [ ]:
p.name

Now suppose we wan't to disallow setting an empty string or `None` for the name. Also, we'll require the name to be a string.

To do that we are going to create an instance method that will handle the logic and setting of the value. We also create an instance method to retrieve the attribute value.

We'll use `_name` as the instance variable to store the name.

In [ ]:
class Person:
    def __init__(self, name):
        self.set_name(name)
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if isinstance(value, str) and len(value.strip()) > 0:
            # this is valid
            self._name = value.strip()
        else:
            raise ValueError('name must be a non-empty string')

In [ ]:
p = Person('Alex')

In [ ]:
try:
    p.set_name(100)
except ValueError as ex:
    print(ex)

In [ ]:
p.set_name('Eric')

In [ ]:
p.get_name()

Of course, our users can still manipulate the atribute directly if they want by using the "private" attribute `_name`. You can't stop anyone from doing this in Python - they should know better than to do that, but we're all good programmers, and know what and what not to do!

The way we set up our initializer, the validation will work too:

In [ ]:
try:
    p = Person('')
except ValueError as ex:
    print(ex)

So this works, but it's a bit of pain to use the method names. So let's turn this into a property instead. We start with the class we just had and tweak it a bit:

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name  # note how we are actually setting the value for name using the property!
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if isinstance(value, str) and len(value.strip()) > 0:
            # this is valid
            self._name = value.strip()
        else:
            raise ValueError('name must be a non-empty string')
            
    name = property(fget=get_name, fset=set_name)

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.name = 'Eric'

In [ ]:
try:
    p.name = None
except ValueError as ex:
    print(ex)

So now we have the benefit of using accessor methods, without having to call the methods explicitly.

In fact, even `getattr` and `setattr` will work the same way:

In [ ]:
setattr(p, 'name', 'John')  # or p.name = 'John'

In [ ]:
getattr(p, 'name')  # or simply p.name

Now let's examine the instance dictionary:

In [ ]:
p.__dict__

You'll see we can find the underlying "private" attribute we are using to store the name. But the property itself (`name`) is not in the dictionary.

The property was defined in the class:

In [ ]:
Person.__dict__

And you can see it's type is `property`.

So when we type `p.name` or `p.name = value`, Python recognizes that `'name` is a `property` and therefore uses the accessor methods. (How it does we'll see later when we study descriptors).

What's interesting is that even if we muck around with the instance dictionary, Python does not get confused - (and as usual in Python, just because you **can** do something does not mean you **should**!)

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.__dict__

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        print('getting name')
        return self._name
    
    def set_name(self, value):
        print('setting name')
        self._name = value
        
    def del_name(self):
        print('deleting name')
        del self._name  # or whatever "cleanup" we want to do
        
    name = property(get_name, set_name, del_name)

In [ ]:
p = Person('Alex')

In [ ]:
p.__dict__

In [ ]:
p.name

In [ ]:
p.name = 'Eric'

In [ ]:
p.__dict__

In [ ]:
del p.name

In [ ]:
p.__dict__

Now, the property still exists (since it is defined in the class) - all we did was remove the underlying value for the property (the `_name` instance attribute):

In [ ]:
try:
    p.name
except AttributeError as ex:
    print(ex)

As you can see the issue is that the getter function is trying to find `_name` in the attribute, which no longer exists. So the getter and setter still exist (i.e. the property still exists), so we can still assign to it:

In [ ]:
p.name = 'Alex'

In [ ]:
p.name

The last param in `property` is just a docstring. So we could do this:

In [ ]:
class Person:
    """This is a Person object"""
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
        
    name = property(get_name, set_name, doc="The person's name.")

In [ ]:
p = Person('Alex')

In [ ]:
help(Person.name)

In [ ]:
help(Person)